# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score

import warnings 
warnings.filterwarnings('ignore')

In [2]:
# load data from database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/disaster_response.db')
df = pd.read_sql_table('features', engine) 
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Lemmatization
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]

    # Stemming
    # stemmed = [PorterStemmer().stem(w) for w in lemmed]
    
    return lemmed
    

In [4]:
# test out function
for message in X.sample(5):
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

A series of seismic tremors under Lake Natron in Tanzania - close to the active volcano Ol Donyo Lengai located some 100 km northwest of Arusha - sent violent tremors across the border to Kenya and was felt as far as Nakuru, Nairobi, Mombasa and parts of Central and Eastern provinces.
['series', 'seismic', 'tremor', 'lake', 'natron', 'tanzania', 'close', 'active', 'volcano', 'ol', 'donyo', 'lengai', 'located', '100', 'km', 'northwest', 'arusha', 'sent', 'violent', 'tremor', 'across', 'border', 'kenya', 'felt', 'far', 'nakuru', 'nairobi', 'mombasa', 'part', 'central', 'eastern', 'province'] 

The prices have doubled for food in Mole Saint Nicolas. A cup of Corn is 45 Gdes now.
['price', 'doubled', 'food', 'mole', 'saint', 'nicolas', 'cup', 'corn', '45', 'gdes'] 

Haiti And Earthquake Disaster! Watch Haiti and Earthquake Pieces Left BehindHaiti and EarthquakeHaiti and Eart. http bit.ly 5XXO9q
['haiti', 'earthquake', 'disaster', 'watch', 'haiti', 'earthquake', 'piece', 'left', 'behindhait

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# build pipeline
pipeline = Pipeline([ 
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(verbose=3)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# evaluate model
from sklearn.metrics import multilabel_confusion_matrix

for i, col in enumerate(y_test.columns):
    print(f'label:',col)
    print(classification_report(y_test[col], y_pred[:, i]))
    print('Accuracy: {:.2f}'.format(accuracy_score(y_test[col], y_pred[:, i])))
    print('F1 Score: {:.2f}'.format(f1_score(y_test[col], y_pred[:, i], average='weighted')))
    print()
    print('Confusion Matrix:\n ', multilabel_confusion_matrix(y_test[col], y_pred[:, i]))
    print('------------------------------------------------------')

In [ ]:
# Calculate metrics for each column
from sklearn.metrics import multilabel_confusion_matrix

# Initialize lists to store metrics
cols = []
accuracies = []
tps = []
tns = []
fps = []
fns = []

# Calculate metrics for each column
for i, col in enumerate(y_test.columns):
    mcm = multilabel_confusion_matrix(y_test[col], y_pred[:, i])
    for j in range(mcm.shape[0]):
        tn, fp, fn, tp = mcm[j].ravel()
        accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    # Append metrics to lists
    cols.append(f'{col}')
    accuracies.append(accuracy)
    tps.append(tp)
    tns.append(tn)
    fps.append(fp)
    fns.append(fn)

# Create DataFrame
df = pd.DataFrame({
    'Feature': cols,
    'Accuracy': accuracies,
    'TP': tps,
    'TN': tns,
    'FP': fps,
    'FN': fns
})


df.sort_values(by='FN', ascending=False)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# use grid search to find better parameters
from sklearn.model_selection import GridSearchCV
from joblib import parallel_backend

parameters = {
    'clf__estimator__n_estimators': [100, 200],
    'clf__estimator__max_depth': [10, 50]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, cv=5)

with parallel_backend('threading', n_jobs=8):
    cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# plot the training and test accuracy 
from matplotlib import pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(cv.cv_results_['mean_test_score'], label='Test Accuracy')
plt.plot(cv.cv_results_['mean_train_score'], label='Train Accuracy')
plt.legend()
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Train and Test Accuracy')
plt.show()


In [ ]:
# before testing for accuracy, we will make predictions on the test set
y_pred = cv.predict(X_test)


In [ ]:
# show accuracy, precision and recall
for i, col in enumerate(y_test.columns):
    print(f'label :',col)
    print()
    print(classification_report(y_test[col], y_pred[:, i]))
    print('_________________________________________________________')
    print()


In [ ]:
# set pareters to best parameters from grid search
pipeline.set_params(**cv.best_params_)

# fit the model
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)
for i, col in enumerate(y_test.columns):
    print(f'label:',col)
    print(classification_report(y_test[col], y_pred[:, i]))
    print('Accuracy: {:.2f}'.format(accuracy_score(y_test[col], y_pred[:, i])))
    print('F1 Score: {:.2f}'.format(f1_score(y_test[col], y_pred[:, i], average='weighted')))
    print()
    print('Confusion Matrix:\n ', multilabel_confusion_matrix(y_test[col], y_pred[:, i]))
    print('------------------------------------------------------')

In [ ]:
# create dataframe with metrics
cols = []
accuracies = []
tps = []
tns = []
fps = []
fns = []

for i, col in enumerate(y_test.columns):
    mcm = multilabel_confusion_matrix(y_test[col], y_pred[:, i])
    for j in range(mcm.shape[0]):
        tn, fp, fn, tp = mcm[j].ravel()
        accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    cols.append(f'{col}')
    accuracies.append(accuracy)
    tps.append(tp)
    tns.append(tn)
    fps.append(fp)
    fns.append(fn)

df = pd.DataFrame({
    'Feature': cols,
    'Accuracy': accuracies,
    'TP': tps,
    'TN': tns,
    'FP': fps,
    'FN': fns
})

df.sort_values(by='FN', ascending=True)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# use the XGBoost classifier for multiclass
from xgboost import XGBClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(XGBClassifier(
        objective='multi:softmax',
        num_class=36,
        n_jobs=8,
        learning_rate=0.1,
        n_estimators=300
    )))
])

pipeline.fit(X_train, y_train)

### 9. Export your model as a pickle file

In [ ]:
# export your model as a pickel file
import joblib
with open('classifier.pkl', 'wb') as file:
    joblib.dump(pipeline, file, compress=5)

# load model


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.